In [1]:
%matplotlib inline


PyTorch: Tensor和autograd
-------------------------------

还是和前面一样实现一个全连接的网络，只有一个隐层而且没有bias，使用欧氏距离作为损失函数。

这个实现使用PyTorch的Tensor来计算前向阶段，然后使用PyTorch的autograd来自动帮我们反向计算梯度。


PyTorch的Tensor代表了计算图中的一个节点。如果``x``是一个Tensor并且``x.requires_grad=True``，
那么``x.grad``这个Tensor会保存某个scalar(通常是loss)对``x``的梯度。



In [2]:
import torch

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # 如果有GPU可以注释掉这行

# N是batch size；D_in是输入大小
# H是隐层的大小；D_out是输出大小。
N, D_in, H, D_out = 64, 1000, 100, 10

# 创建随机的Tensor作为输入和输出
# 输入和输出需要的requires_grad=False(默认，因为我们不需要计算loss对它们的梯度。
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 创建weight的Tensor，需要设置requires_grad=True 
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # Forward阶段: mm实现矩阵乘法，但是它不支持broadcasting。如果需要broadcasting，可以使用matmul
    # clamp本来的用途是把值clamp到指定的范围，这里实现ReLU。 
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # pow(2)实现平方计算。 
    # loss.item()得到这个tensor的值。也可以直接打印loss，这会打印很多附加信息。
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())

    # 使用autograd进行反向计算。它会计算loss对所有对它有影响的requires_grad=True的Tensor的梯度。
    
    loss.backward()

    # 手动使用梯度下降更新参数。一定要把更新的代码放到torch.no_grad()里
    # 否则下面的更新也会计算梯度。后面我们会使用torch.optim.SGD，它会帮我们管理这些用于更新梯度的计算。
    
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 手动把梯度清零 
        w1.grad.zero_()
        w2.grad.zero_()

0 42577276.0
1 50359256.0
2 65796516.0
3 67674760.0
4 42845276.0
5 15258689.0
6 4446020.5
7 2015284.25
8 1388696.375
9 1098846.25
10 901679.8125
11 750083.75
12 629551.9375
13 532398.6875
14 453381.90625
15 388429.625
16 334606.5
17 289705.4375
18 252044.1875
19 220220.96875
20 193169.21875
21 170088.75
22 150274.703125
23 133194.75
24 118415.890625
25 105580.4921875
26 94393.828125
27 84614.8984375
28 76035.4375
29 68481.7734375
30 61815.71484375
31 55911.49609375
32 50669.3203125
33 45999.05859375
34 41832.07421875
35 38103.828125
36 34761.57421875
37 31763.423828125
38 29069.8984375
39 26642.462890625
40 24449.857421875
41 22465.3515625
42 20666.79296875
43 19033.759765625
44 17548.796875
45 16196.2939453125
46 14963.87890625
47 13838.119140625
48 12808.517578125
49 11865.458984375
50 11001.185546875
51 10208.166015625
52 9479.0048828125
53 8808.4482421875
54 8191.31982421875
55 7622.75634765625
56 7098.291015625
57 6613.82666015625
58 6166.24951171875
59 5752.46044921875
60 5369.38